1. Data Exploration

First we're going import the required libraries, then we obtain and display the data from the csv file

In [63]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
Not_available = ['Not specified', 'Not Priced']
df = pd.read_csv('car_data.csv', sep = ',', index_col= 'id', na_values=Not_available)
df

,Model,Year,Status,Mileage,Price,MSRP
id,,,,,,
0,2022 Acura TLX A-Spec,2022,New,Not available,"$49,445","MSRP $49,445"
1,2023 Acura RDX A-Spec,2023,New,Not available,"$50,895",NaN
2,2023 Acura TLX Type S,2023,New,Not available,"$57,745",NaN
3,2023 Acura TLX Type S,2023,New,Not available,"$57,545",NaN
4,2019 Acura MDX Sport Hybrid 3.0L w/Technology ...,2019,Used,"32,675 mi.","$40,990",$600 price drop
...,...,...,...,...,...,...
10005,2023 Volkswagen Atlas 3.6L SE w/Technology,2023,New,Not available,"$47,346",NaN
10006,2023 Volkswagen Taos 1.5T SE,2023,New,Not available,"$30,895",NaN
10007,2012 Volkswagen Beetle,2012,Used,"100,395 mi.","$9,994",$252 price drop


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115762 entries, 0 to 10009
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Model    115762 non-null  object
 1   Year     115762 non-null  int64 
 2   Status   115762 non-null  object
 3   Mileage  115762 non-null  object
 4   Price    115110 non-null  object
 5   MSRP     43665 non-null   object
dtypes: int64(1), object(5)
memory usage: 6.2+ MB


We check our dataset for any amount of missing values.

In [66]:
df.isna().sum().sort_values(ascending = False)


MSRP       72097
Price        652
Model          0
Year           0
Status         0
Mileage        0
dtype: int64

At first glance it seems that there are no null values on the Mileage variable so we'll examine it deeply

In [67]:
df["Mileage"].value_counts()

Not available    47868
310 mi.            101
23,000 mi.          19
1,000 mi.           18
35,000 mi.          17
                 ...  
25,796 mi.           1
51,830 mi.           1
34,991 mi.           1
62,460 mi.           1
125,757 mi.          1
Name: Mileage, Length: 49282, dtype: int64

It turns out that out of 49282 entries, most of them are "Not available", in some cases it's because of the car being brand new, so we make sure to check how many of the "Not Available" values match the new cars

In [68]:
df[df["Mileage"]=="Not available"]["Status"]

id
0        New
1        New
2        New
3        New
5        New
        ... 
10001    New
10004    New
10005    New
10006    New
10008    New
Name: Status, Length: 47868, dtype: object

We find out that only 10008 entries have a New car matched to them, that means that there are 37860 entries that are null values

2. Data Cleaning

Since the ammount of missing values on the MSRP and Mileage columns surpasses the 10%, the following needs to be done:
- Drop the MSRP column
- Drop the Mileage column 

As for the Price, we'll convert it into a float in order to replace missing values.

In [69]:
df.drop(columns = ['Mileage','MSRP'], inplace = True)
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

df['Price'] = df['Price'].apply(clean_currency).astype('float')
df['Price'] = df['Price'].fillna(df['Price'].mode()[0])

df.info()
df.isna().sum().sort_values(ascending = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115762 entries, 0 to 10009
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Model   115762 non-null  object 
 1   Year    115762 non-null  int64  
 2   Status  115762 non-null  object 
 3   Price   115762 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.4+ MB


Model     0
Year      0
Status    0
Price     0
dtype: int64

With the data cleaning and sample obtained, we are going to split the year, brand and model separately

In [70]:
df["Model"].value_counts()

2023 Mercedes-Benz S-Class S 580 4MATIC    916
2023 Mercedes-Benz S-Class S 500 4MATIC    800
2023 Mercedes-Benz GLE 450 AWD 4MATIC      700
2018 Tesla Model 3 Long Range              548
2022 Mercedes-Benz S-Class S 580 4MATIC    500
                                          ... 
2011 INFINITI QX56 7-passenger               1
2015 INFINITI QX60 Base (CVT)                1
2022 INFINITI QX80 LUXE                      1
2010 INFINITI M35                            1
2011 Volkswagen Tiguan SE                    1
Name: Model, Length: 9216, dtype: int64

In [71]:
def brand_extraction(model):
  # We split the model value in diferent values, for example:
  # ["2023","Mercedes-Benz","S-Class","S","580","4MATIC"]
  m = model.split(" ")
  brand = m[1] ## we take the brand from the split 
  return brand

df["Brand"] = df["Model"].apply(brand_extraction)


In [72]:
def model_extraction(model):
  m = model.split(" ")
  return " ".join(m[2:4])

df["Model"] = df["Model"].apply(model_extraction)

We'll delete duplicate values in order to asess the models themselves

In [73]:
df = df.drop_duplicates()

In [74]:
df.head()

,Model,Year,Status,Price,Brand
id,,,,,
0,TLX A-Spec,2022,New,49445.0,Acura
1,RDX A-Spec,2023,New,50895.0,Acura
2,TLX Type,2023,New,57745.0,Acura
3,TLX Type,2023,New,57545.0,Acura
4,MDX Sport,2019,Used,40990.0,Acura


In [75]:
df["Status"].value_counts()

Used                    55538
New                     28699
Porsche Certified        1804
Acura Certified          1012
INFINITI Certified        839
Volkswagen Certified      772
BMW Certified             561
Dodge Certified           377
Chevrolet Certified       206
Toyota Certified          183
Ford Certified             29
Name: Status, dtype: int64

Since there are a lot of certified values, we'll simplify it down to three

In [76]:
def change_status(status):
  if("Certified" in status):
    return "Certified"
  return status
df["Status"] = df["Status"].apply(change_status)

C:\Users\Boss7\AppData\Local\Temp\ipykernel_28964\864128867.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Status"] = df["Status"].apply(change_status)


In [79]:
new_col_order = ["Brand","Model","Year","Status","Price"]
df= df.reindex(columns = new_col_order)

In [80]:
df.head()

,Brand,Model,Year,Status,Price
id,,,,,
0,Acura,TLX A-Spec,2022,New,49445.0
1,Acura,RDX A-Spec,2023,New,50895.0
2,Acura,TLX Type,2023,New,57745.0
3,Acura,TLX Type,2023,New,57545.0
4,Acura,MDX Sport,2019,Used,40990.0
